# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_6914c0dd24ec8191a84a2cd6b746a59d


### Upload Files

In [6]:
with open('tweet_text (1).json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-5ZmWgBW5hdBo7vspaKE3vb


### Attach File to Vector Store

In [7]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-5ZmWgBW5hdBo7vspaKE3vb


### Query the Vector Store

In [8]:
query = "the latest development in generativeAI"

In [9]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

## OpenAI Response API

### Simple Response

In [10]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [11]:
display(Markdown(simple_response.output_text))

As of the latest updates, generative AI continues to advance across various domains. Some key developments include:

1. **Larger Language Models**: Companies are developing increasingly larger models, with improved capabilities in natural language understanding, generation, and conversation. These models are being fine-tuned for specific tasks, leading to more accurate and context-aware interactions.

2. **Enhanced Multimodal Models**: Advances are being made in models that can process and generate multiple types of data, such as text, images, and audio, allowing for more sophisticated applications like text-to-image generation, video creation, and enhanced data synthesis.

3. **Responsible AI and Ethics**: There's a stronger focus on ensuring generative AI is used responsibly, with efforts to mitigate biases, improve transparency, and ensure ethical applications. Frameworks and guidelines are being developed to guide the responsible deployment of these technologies.

4. **Real-time Applications**: Generative AI is increasingly used in real-time applications, such as virtual reality (VR), augmented reality (AR), and interactive storytelling, enhancing user engagement and personalization.

5. **Industry-Specific Solutions**: Companies are deploying industry-specific generative AI solutions in fields like healthcare, finance, and entertainment, providing tailored tools and models that address unique challenges and workflows within these sectors.

6. **Collaboration and Open Source**: There is a growing trend towards collaboration and open-sourcing of AI models and tools, which is helping to democratize access to advanced AI technologies and drive innovation.

These developments highlight the rapid pace of innovation in generative AI, with applications expanding into new areas and becoming more integrated into everyday technologies.

### File Search Response

In [12]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [13]:
display(Markdown(file_search_response.output_text))


Recent developments in generative AI include:

1. **OpenAI's Sora2**: This tool allows users to create cinematic videos from a prompt, incorporating audio, physics, and cameos, offering endless creative possibilities.

2. **Generative AI in Business**: It's being used across various sectors, from content generation to design systems, reshaping how businesses innovate.

3. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis using generative AI to assess risk and boost resilience.

4. **Market Growth**: The global generative AI market is expected to reach $1.18 billion this year, highlighting its rapid adoption and growth.

These advancements demonstrate the expanding role of generative AI in creative, business, and industrial applications.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [14]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [15]:
display(Markdown(web_search_response.output_text))

Here’s an **in-depth, highly up-to-date overview** of the most significant recent developments in generative AI as of **Wednesday, November 12, 2025**. This response is structured into the latest **model releases**, **technological breakthroughs across modalities**, **industry applications and adoption**, and **global dynamics and implications**. Each section includes precise dates and robust citations.

---

##  Recent Model Releases and Enhancements

- **OpenAI GPT‑5**  
  Released on **August 7, 2025**, GPT‑5 is OpenAI’s most advanced multimodal foundational model, integrating reasoning and non-reasoning capabilities into a unified interface. It powers ChatGPT, Microsoft Copilot, and is accessible via the OpenAI API ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai)).

- **OpenAI GPT‑4.1 & o4‑mini**  
  GPT‑4.1 debuted on **April 14, 2025**, offering a new lineup—GPT‑4.1, GPT‑4.1 mini, and GPT‑4.1 nano—with enhanced accessibility across ChatGPT Plus and Pro plans ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-4.1?utm_source=openai)). On the same day, OpenAI also released *o4‑mini*, a reasoning-focused multimodal lightweight model capable of processing text and images, including recognition of whiteboard sketches and documents ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI_o4-mini?utm_source=openai)).

- **Google's Gemini and Nano Banana**  
  Google has expanded its Gemini AI family significantly in 2025.  
  - *Gemini 2.5 Pro* introduced advanced reasoning, coding capabilities, and a “Deep Think” mode, along with support for native audio output (available from **March–June 2025**) ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai)).  
  - The image model *Nano Banana* (Gemini 2.5 Flash Image), launched **August 26, 2025**, gained viral traction for its photorealistic, 3D-figurine image generation, subject consistency features, and SynthID watermarking ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai)).

- **Other notable model releases**  
  - **Baidu’s Ernie 4.5 and Ernie X1**, both multimodal models claimed to outperform competitors, with *Ernie 4.5* slated for open-source release starting **June 30, 2025** ([globenewswire.com](https://www.globenewswire.com/news-release/2025/04/17/3063915/0/en/Applied-Generative-AI-Course-Launched-by-Interview-Kickstart-2025-Best-GenAI-Course-With-Agentic-AI-Projects-For-Top-AI-Jobs-at-Google-Meta-Netflix-Microsoft-OpenAI-Nvidia.html?utm_source=openai)).  
  - **Alibaba’s Qwen3**, a family of open-source language models (0.6B to 32B parameters), including Mixture-of-Experts variants and dual “thinking/non-thinking” modes, released in **April 2025** ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-digest-a-wave-of-notable-ai-model-launches?utm_source=openai)).  
  - **Open-source DeepSeek Prover v2**, built for generating mathematical proofs in Lean 4 by decomposing complex problems into sub-goals ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-digest-a-wave-of-notable-ai-model-launches?utm_source=openai)).

---

##  Breakthroughs in Multimodal and Creative Interfaces

- **Veo 3 (Google)**  
  Released in **June 2025**, Veo 3 is a text-to-video model capable of generating video with synchronized audio—including ambient sounds and dialogue—eliminating substantial traditional post-production work ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai)).

- **Sora 2 and Social App (OpenAI)**  
  Announced on **September 30, 2025**, *Sora 2* brings enhanced text-to-video with audio and multi-shot consistency. It is complemented by the *Sora app*, enabling users to generate short-form AI video clips and share them socially—marking a major leap toward consumer-facing, social generative platforms ([champaignmagazine.com](https://champaignmagazine.com/2025/10/05/ai-by-ai-weekly-top-5-september-29-october-5-2025/?utm_source=openai)).

- **Midjourney v7, Runway Gen‑4, Stability AI Stable Audio 2.0, and others**  
  Additional advancements in 2025 include:
  - Midjourney v7 with 3D (“NeRF-like”) support and text-to-video ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai)).
  - Runway’s Gen-4 featuring “Character Lock” for consistent identity in video sequences (April 2025) ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai)).
  - Stability AI’s *Stable Audio 2.0* (March 2025) offering longer, emotionally nuanced music generation ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai)).
  - Microsoft’s *Phi-4* (February 2025), natively integrating vision, audio, and text in a unified multimodal architecture ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai)).
  - Tencent’s *Hunyuan3D* (January 2025) setting new benchmarks for clean, realistic 3D model outputs ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai)).

- **Google DeepMind’s Gemini Diffusion Model**  
  An experimental model, *Gemini Diffusion* applies diffusion techniques to accelerate text generation, achieving speeds up to roughly 1,479 tokens per second—far exceeding Gemini 2.5 Flash (~400 tps) and GPT‑4o (~150 tps) ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-digest-a-wave-of-notable-ai-model-launches?utm_source=openai)).

- **DeepMind’s Genie 3**  
  Launched **August 5, 2025**, this model generates interactive 3D environments from text prompts—complete with memory and physics—allowing real-time navigation at 24 FPS and 720p resolution ([scrumlaunch.com](https://www.scrumlaunch.com/blog/top-5-breakthrough-ai-launches-summer-2025?utm_source=openai)).

---

##  Adoption, Integration, and Real-world Applications

- **AI Adoption Rates**  
  As of late 2025, **78% of organizations** are using AI in at least one business function, up steeply from 55% a year prior, signifying AI's shift from experimental to essential ([netguru.com](https://www.netguru.com/blog/ai-adoption-statistics?utm_source=openai)).

- **Enterprise Practices & Scaling**  
  According to the latest McKinsey Global Survey (published last week), **over one-third** of high-performing organizations now allocate more than **20%** of their digital budgets to AI. About **75%** of these leaders report they have scaled AI beyond pilot stages, compared to just **one-third** of other firms ([mckinsey.com](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai?utm_source=openai)).

- **Generative AI in Contact Centers**  
  In 2025, contact centers are evolving from pilots to large-scale deployment. **83% of business leaders** plan to use generative AI for proactive customer outreach, indicating a shift toward predictive, personalized engagement strategies ([cxtoday.com](https://www.cxtoday.com/contact-center/generative-ai-in-the-contact-center-whats-new-in-2025/?utm_source=openai)).

- **Journalism and Media Innovation**  
  *TIME* unveiled the **TIME AI Agent** just **two days ago** (November 10, 2025). Co-developed with Scale AI, this interactive platform supports personalized summaries, audio reports, translations, and conversational exploration of journalism, combining editorial standards with generative intelligence ([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai)).

- **Ethical & Strategic Oversight**  
  At a 2025 industry forum, leaders emphasized balancing AI’s value and risk—encouraging critical thinking to responsibly guide AI deployments in professional services ([thomsonreuters.com](https://www.thomsonreuters.com/en-us/posts/technology/emerging-technology-generative-ai-forum-ethical-ai-adoption/?utm_source=openai)).

---

##  Geopolitics & Industry Trends

- **Supply Chain Shifts**  
  Tariffs and export controls on advanced chips (e.g., NVIDIA’s A100, H100) are disrupting generative AI supply chains. Companies are relocating supply bases to Vietnam, Mexico, and India, while Chinese firms intensify domestic chip development. This fuels a divergence between Western-supported AI ecosystems and parallel ecosystems emerging in China, Russia, and the Global South ([globenewswire.com](https://www.globenewswire.com/news-release/2025/09/16/3150581/28124/en/Generative-AI-Market-Report-Highlights-US-Tariffs-Rising-Costs-and-Global-AI-Ecosystem-Shifts.html?utm_source=openai)).

---

##  Summary: Key Takeaways

- **New model launches**: GPT‑5 (Aug 2025), GPT‑4.1 (Apr 2025), o4‑mini, Gemini 2.5 (Pro & Flash), Nano Banana (Aug 2025), Baidu’s Ernie 4.5/X1, Alibaba Qwen3.
- **Modal advances**: Sora 2 with social video; Veo 3 with audio-video; interactive 3D via Genie 3; real-time text via Gemini Diffusion.
- **Adoption leap**: AI embedded across industries, with clear ROI and scale strategies.
- **Use-case innovation**: Contact center automation, AI-powered journalism, ethical AI frameworks.
- **Geopolitical context**: Supply chain reconfiguration and ecosystem bifurcation.

If you’d like to explore a specific model, application, or trend in more detail—such as GPT‑5’s unique capabilities, Sora 2's content moderation challenges, or the geopolitical dynamics shaping AI research—feel free to ask!


### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [17]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here’s an **in-depth, highly up-to-date overview** of the most significant recent developments in ge

### Continue Query with Web Search

In [18]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [19]:
display(Markdown(continue_search_response.output_text))

Below are several **recent and noteworthy news developments** in generative AI as of **Wednesday, November 12, 2025**. Each summary includes key details, publication dates, and citations for transparency and verification.

---

## AWS Introduces Generative AI-Focused Training Tools (Nov 12, 2025)
- AWS unveiled a suite of new educational tools on **November 12, 2025**, including the **Skill Builder Meeting Simulator**, **Cohorts Studio**, microcredentials, and a **Professional Generative AI Developer certification** (beta exam registration opens November 18).
- These offerings provide AI-powered communication skills training, realistic workplace scenario simulations, and credentialing in serverless computing and agentic systems—addressing both technical skills and the ability to explain AI concepts to diverse stakeholders. ([aboutamazon.com](https://www.aboutamazon.com/news/aws/aws-ai-certification-learning-tools-skills-development?utm_source=openai))

---

## Georgia Tech Advances Generative AI Research with Microsoft Support (Nov 12, 2025)
- Also published **today**, researchers at Georgia Tech's Cloud Hub (IDEaS) are expanding GenAI research thanks to a generous funding contribution from **Microsoft**.
- The initiative supports projects exploring foundational generative AI models and innovative real-world applications. ([newswise.com](https://www.newswise.com/articles/georgia-tech-cloud-hub-advances-generative-ai-research-with-microsoft-support?utm_source=openai))

---

## Insilico Medicine to Showcase Generative AI for Pulmonary Fibrosis (Nov 13–15, 2025)
- Insilico Medicine announced it will present its latest generative AI platform and breakthroughs in AI-driven pulmonary fibrosis research at the **Pulmonary Fibrosis Foundation Summit in Chicago from November 13 to 15, 2025**.
- This highlights the growing intersection of generative AI and biomedical innovation. ([bioengineer.org](https://bioengineer.org/insilico-medicine-to-unveil-generative-ai-platform-and-cutting-edge-ai-driven-pulmonary-fibrosis-research-at-pff-summit-2025-in-chicago/?utm_source=openai))

---

## The “Illusion” of Social AI Explored in New Research (Nov 10, 2025)
- On **November 10, 2025**, a study examined whether interactions with chatbots like ChatGPT, Replika, or Gemini foster genuine relationships—or just the “illusion” of social connection.
- This research prompts reflection on how people form emotional bonds with AI, raising questions about authenticity and human–machine interaction. ([lboro.ac.uk](https://www.lboro.ac.uk/news-events/news/2025/november/illusion-of-ai-interaction/?utm_source=openai))

---

## Kong Releases Insomnia 12 with AI-Enhanced API Development (Nov 4, 2025)
- On **November 4, 2025**, Kong released **Insomnia 12**, an upgrade to its open-source API development platform.
- It integrates AI mock servers, native Model Context Protocol (MCP) clients, and AI-powered commit suggestions to streamline the development and testing of APIs and MCP servers. ([infoworld.com](https://www.infoworld.com/article/4086891/kong-insomnia-12-bolsters-ai-mcp-tools.html?utm_source=openai))

---

## TIME Launches the “TIME AI Agent” (Nov 10, 2025)
- Just **two days ago**, TIME launched the **TIME AI Agent**, co-developed with Scale AI.
- This unified generative AI platform enhances journalistic experiences with personalized summaries, audio reports, multilingual translations, and interactive exploration—while upholding editorial integrity through moderation and data governance. ([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai))

---

## Innodata Launches AI-Focused Federal Business Unit (Nov 7, 2025)
- On **November 7, 2025**, Innodata unveiled a **Federal Business Unit** tailored for U.S. government needs.
- The unit offers AI data engineering, generative AI, and agentic AI solutions for defense, intelligence, and civilian agencies—underscoring the expanding role of generative AI in public sector services. ([tradingview.com](https://www.tradingview.com/news/reuters.com%2C2025-11-07%3Anewsml_RSG5671Ga%3A0-rcs-innodata-inc-innodata-launches-federal-business-unit/?utm_source=openai))

---

## Sydney Informatics Hub Empowers AI-Based Research (Nov 4, 2025)
- Last week, the **University of Sydney’s Sydney Informatics Hub** launched support initiatives to help Australian researchers navigate the opportunities and challenges presented by generative AI.
- This effort aims to foster both innovation and responsible adoption within the academic community. ([sydney.edu.au](https://www.sydney.edu.au/news-opinion/news/2025/11/04/sydney-informatics-hub-unlocks-generative-ai-for-research.html?utm_source=openai))

---

## EU Mulls Softening Its Landmark AI Act Amid Tech Lobbying (Nov 7, 2025)
- Published **five days ago**, the **Financial Times** reports that the **European Commission** is considering softening parts of its **AI Act**, effective since August 2024.
- Proposed concessions include a one-year compliance grace period and deferred enforcement of transparency fines until 2027—motivated by Big Tech lobbying and U.S. trade pressure. These changes will be discussed around **November 19, 2025**. ([ft.com](https://www.ft.com/content/af6c6dbe-ce63-47cc-8923-8bce4007f6e1?utm_source=openai))

---

## AI Reshaping Stock Market Trends (Nov 12, 2025)
- Also released **today**, a report highlights how AI-related optimism and rapid technological progress are fueling the stock market’s current rally.
- Investor sentiment remains bullish, though tempered by caution over sustainability and hype-driven market dynamics. ([markets.financialcontent.com](https://markets.financialcontent.com/wral/article/marketminute-2025-11-12-the-ai-ascent-how-hopes-and-hype-are-reshaping-the-2025-stock-market?utm_source=openai))

---

## AI-Generated Country Singer “Breaking Rust” Tops Charts (Nov 10, 2025)
- The AI-generated country music artist **Breaking Rust** has become a sensation. Its single “Walk My Walk” reached **No. 1 on the Billboard Country Digital Song Sales chart**, marking a first for an AI artist.
- The synthetic act, created by Aubierre Rivaldo Taylor, has also triggered debates over authenticity, chart legitimacy, and the evolving role of AI in the creative industries. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Breaking_Rust?utm_source=openai))

---

### Summary: Key Themes and Takeaways

- **Education & Workforce**: AWS’s AI-focused learning tools (Nov 12) and Georgia Tech’s research boost show investment in AI skills and innovation.
- **Industry & Enterprise**: Innodata expands generative AI into federal solutions (Nov 7), while Kong streamlines API development with AI enhancements (Nov 4).
- **Media & Culture**: TIME’s AI Agent reimagines journalism (Nov 10), and Breaking Rust challenges music industry norms (Nov 10).
- **Global Regulation & Policy**: The EU may ease AI Act enforcement (Nov 7), affecting compliance timelines.
- **Science & Healthcare**: Insilico’s AI advances spotlight biomedical applications (Nov 13–15), and Australian researchers gain AI support (Nov 4).
- **Market Impact**: AI is visibly influencing investor behavior and market valuations (Nov 12).

Let me know if you'd like to dive deeper into any of these developments—whether it's the policy implications of the EU’s AI Act, the technical capabilities of TIME’s AI Agent, or the ethics behind AI-generated artistry.

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [20]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [21]:
display(Markdown(combined_search_response.output_text))

Here’s a comprehensive and up-to-date overview of the **latest developments in generative AI** as of November 12, 2025. This analysis draws on recent news, model releases, market trends, and research breakthroughs, with citations for each key point.

---

##  Major Model Releases and Innovations

- **OpenAI GPT‑5**  
  Released on **August 7, 2025**, GPT‑5 is a multimodal foundation model that unifies reasoning and non-reasoning capabilities under a single interface. It is accessible via ChatGPT, Microsoft Copilot, and the OpenAI API, and achieved state-of-the-art performance across benchmarks at launch ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai)).

- **OpenAI o4‑mini**  
  Launched on **April 16, 2025**, this lightweight reasoning model supports both text and image inputs, including whiteboard sketch analysis. A higher-accuracy variant, o4‑mini‑high, is available to paid-tier users ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI_o4-mini?utm_source=openai)).

- **OpenAI GPT‑4.1**  
  Released on **April 14, 2025**, GPT‑4.1 (along with mini and nano variants) offers improved coding capabilities and is available to ChatGPT Plus and Pro subscribers ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-4.1?utm_source=openai)).

- **Google’s Gemini Family & “Nano Banana”**  
  Google’s Gemini models have seen rapid evolution:
  - **Gemini 2.5 Pro** (March 25, 2025) introduced advanced reasoning, coding, and a “thinking model” with a 1 million token context window ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai)).
  - **Nano Banana** (Gemini 2.5 Flash Image), launched on **August 26, 2025**, is a viral image generation/editing tool known for photorealistic 3D figurine-style outputs, multi-image fusion, and SynthID watermarking. It attracted over 10 million new users and enabled 200 million image edits within weeks ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai)).

- **Other Notable Models**  
  - **GPT‑4.5** (February 2025) enhanced multimodal processing, context length, and latency, serving as a bridge to GPT‑5 ([analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2025/03/generative-ai-launches-of-february/?utm_source=openai)).  
  - **xAI’s Grok 3 & Grok 3 Mini**, **Anthropic’s Claude 3.7 Sonnet**, **Baidu’s Ernie 4.5 and Ernie X1**, and **DeepSeek‑R1** are among other significant releases earlier in 2025 ([analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2025/03/generative-ai-launches-of-february/?utm_source=openai)).

---

##  Emerging Trends & Ecosystem Integration

- **Agentic AI and Autonomous Agents**  
  The shift from AI as copilots to autonomous agents is accelerating. Agentic AI systems can plan, act, and execute workflows independently. The autonomous AI market is projected to reach USD 11.79 billion by 2026, growing at over 40% CAGR ([simplilearn.com](https://www.simplilearn.com/top-technology-trends-and-jobs-article?utm_source=openai)).

- **Enterprise Adoption and Scaling**  
  According to the **2025 McKinsey Global Survey**, 88% of organizations now use AI in at least one business function (up from 78% last year), though only about one-third have scaled AI programs. Agentic AI experimentation is underway in 39% of organizations, with 23% scaling deployments. High performers are more likely to redesign workflows, define human validation processes, and allocate over 20% of digital budgets to AI ([mckinsey.com](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai?utm_source=openai)).

- **Market Growth**  
  The generative AI market experienced triple-digit growth across hardware, foundation models, and development platforms in 2024. AI-related spending is expected to exceed **US$400 billion in 2025** ([businesswire.com](https://www.businesswire.com/news/home/20250825682581/en/Generative-AI-Market-Report-2025-GenAI-Market-Experienced-Triple-digit-growth-Rates-in-All-Three-Major-Segments-Spanning-GenAI-Hardware-Foundation-Models-and-Development-Platforms---ResearchAndMarkets.com?utm_source=openai)).

---

##  Responsible Use & Human-AI Collaboration

- **Responsible Generative AI Playbook**  
  UC Berkeley’s Responsible AI Initiative released a playbook outlining 10 actionable strategies (five for business leaders, five for product managers) to ensure responsible GenAI development and usage ([weforum.org](https://www.weforum.org/stories/2025/06/responsible-generative-ai-product-development-use/?utm_source=openai)).

- **Human Skills Remain Critical**  
  A Harvard Business Review article (September 2025) highlights that as GenAI becomes more integrated, human soft skills—such as problem framing, creativity, and collaboration—are increasingly essential to maximize AI’s value ([hbr.org](https://hbr.org/2025/09/how-gen-ai-could-transform-learning-and-development?utm_source=openai)).

- **Unexpected Productivity Findings**  
  A randomized controlled trial found that experienced open-source developers using early‑2025 AI tools took **19% longer** to complete tasks, suggesting that AI may introduce friction rather than efficiency in some contexts ([metr.org](https://metr.org/blog/2025-07-10-early-2025-ai-experienced-os-dev-study/?utm_source=openai)).

---

##  Recent Breakthroughs & Product Launches (November 2025)

- **OpenAI Sora (Video + Audio)**  
  - **Sora 2**, a multimodal video+audio generation model, was released on **September 30, 2025**, along with a social-style Sora app for short-form AI-generated clips. The launch raised concerns around IP, moderation, and deepfake risks ([champaignmagazine.com](https://champaignmagazine.com/2025/10/05/ai-by-ai-weekly-top-5-september-29-october-5-2025/?utm_source=openai)).  
  - The **Android version of Sora** launched on **November 6, 2025**, achieving nearly **470,000 downloads on day one**, outperforming the iOS debut by 327% ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

- **Google DS STAR**  
  Google introduced **DS STAR**, a multi-agent framework that converts ambiguous business problems into executable Python code, handling mixed-format data (CSV, JSON, Markdown, unstructured text) without human intervention ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

- **Meta Vibes Expansion**  
  Meta’s AI-generated short video platform **Vibes** expanded to Europe via the Meta AI app, offering automated, AI-generated content in a TikTok-like format ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

- **Google Gemini Enhancements**  
  - A preview of **Gemini 3 Pro** appeared on Vertex AI, expected to support a 1-million-token context window ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).  
  - Google also launched a **Deep Research** feature for Gemini, enabling it to generate research reports using data from Gmail, Drive, and Chat ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

- **ClickUp 4.0**  
  ClickUp released version 4.0, featuring a redesigned UI and two AI agents that integrate task management, collaboration, messaging, scheduling, and enterprise search into a unified platform ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

- **Google Veo 3.1 & Sora 2**  
  - **Veo 3.1**, a generative video model, now offers narrative control, character consistency, and integrated audio via the Gemini API and Flow editing app ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai)).  
  - **Sora 2** continues to gain traction as a consumer-facing video+audio generative platform ([champaignmagazine.com](https://champaignmagazine.com/2025/10/05/ai-by-ai-weekly-top-5-september-29-october-5-2025/?utm_source=openai)).

- **Anthropic Claude Haiku 4.5**  
  A compact model delivering flagship-level performance at 4–5× speed and one-third the cost. It supports agentic capabilities and large context windows (200K tokens input, 64K output) ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai)).

---

##  Summary: Key Themes in Generative AI (2025–Nov 2025)

1. **Multimodal and Agentic AI**: Models like GPT‑5, Gemini 2.5 Pro, and Sora 2 are pushing boundaries in reasoning, video/audio generation, and autonomous workflows.
2. **Enterprise Scaling**: Adoption is growing, but scaling remains a challenge. High performers are investing heavily and redesigning workflows to capture value.
3. **Market Expansion**: Generative AI is experiencing explosive growth, with massive investments and market expansion across hardware and software.
4. **Responsible AI & Human-AI Synergy**: Ethical frameworks and human soft skills are increasingly recognized as essential to effective AI deployment.
5. **Rapid Productization**: From DS STAR to Vibes and ClickUp 4.0, generative AI is being embedded into everyday tools and platforms.

---

If you'd like to explore any of these developments in more depth—such as technical details of GPT‑5, enterprise case studies, or ethical frameworks—just let me know!

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [22]:
# create vector store
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print('vector store id: ', vector_store_id)


vector store id:  vs_6914c19d4f008191a22a129c7b879bc6


In [23]:
# upload files to file search
with open('MLB_pitch_tracking.txt', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )
file_id = file.id
print(file_id)

file-T8HSvvNCHnZPNUU6eEDTUP


In [24]:
# attach file to vector source
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )
print(attach_status.id)

file-T8HSvvNCHnZPNUU6eEDTUP


In [25]:
query = "recent technical and usage developments of mlb pitch tracking system"

In [26]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

MLB Pitch-Tracking and Strike-Calling Systems

Technical breakdown:
The Automated Ball‑Strike Sys
 Relevant score: 0.8804389336434647


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [27]:
query = "Using my uploaded notes and the latest web information, summarize the current trends on the mlb pitch tracking system."

In [28]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [29]:
display(Markdown(combined_search_response.output_text))

Here’s a comprehensive, structured summary of the **current trends in MLB pitch‑tracking systems**, combining insights from your uploaded notes (if any) with the latest developments from the web. The focus is on two major areas: **Statcast enhancements** and the **Automated Ball‑Strike System (ABS)**.

---

##  Statcast Enhancements & Pitch‑Tracking Technology

- **Upgraded Camera Infrastructure**  
  MLB’s Statcast system now employs 12 Hawk‑Eye cameras per ballpark. Five of these are high‑frame‑rate units (upgraded from 100 fps to 300 fps in 2023) dedicated to bat and pitch tracking, while the remaining seven focus on player and batted‑ball tracking. This upgrade has increased batted‑ball tracking accuracy from approximately 89% to 99% ([mlb.com](https://www.mlb.com/glossary/statcast?utm_source=openai)).

- **Expanded Metrics and Real‑Time Analytics**  
  Statcast continues to deliver detailed metrics—such as velocity, spin rate, spin direction, pitch movement, exit velocity, launch angle, sprint speed, and more—on a play‑by‑play basis. These data feed into advanced statistics like Outs Above Average and hard‑hit rate, which are now widely used by teams, broadcasters, and fans ([mlb.com](https://www.mlb.com/glossary/statcast?utm_source=openai)).

- **Training Integration with Rapsodo and TrackMan**  
  Teams like the Seattle Mariners are fully integrating pitch‑tracking tools such as Rapsodo and TrackMan into their training routines. These systems provide instant feedback on spin, movement, release point, and stride mechanics, enabling objective coaching and player development ([mlb.com](https://www.mlb.com/news/mariners-fully-embrace-pitch-tracking-data-c303971178?utm_source=openai)).

---

##  Automated Ball‑Strike System (ABS) – “Robot Umpires”

- **Overview and Implementation Timeline**  
  MLB has approved the **Automated Ball‑Strike System (ABS)** for use beginning in the **2026 season**. This system will operate in a **challenge format**, where human umpires make initial calls, and players (pitcher, catcher, or batter) may challenge ball/strike calls ([apnews.com](https://apnews.com/article/d70c6431d1cccfcf7a6e69e3ce47b417?utm_source=openai)).

- **Technology and Infrastructure**  
  ABS relies on **12 Hawk‑Eye cameras** and a **T‑Mobile private 5G network** to track pitches in real time. The strike zone is personalized based on each batter’s certified height, and if any part of the ball touches the zone, it is ruled a strike ([theverge.com](https://www.theverge.com/news/783982/mlb-baseball-robot-umpires-abs?utm_source=openai)).

- **Challenge Mechanics and Rules**  
  Each team receives **two challenges per game**, with an additional challenge granted in extra innings if none remain. Challenges are signaled by tapping the helmet or cap immediately after the call. Successful challenges are retained; unsuccessful ones are lost. Results are displayed via digital graphics in stadiums and broadcasts, with decisions typically rendered in **13–17 seconds** ([apnews.com](https://apnews.com/article/d70c6431d1cccfcf7a6e69e3ce47b417?utm_source=openai)).

- **Testing and Performance Metrics**  
  ABS has undergone extensive testing:
  - **Minor leagues**: since 2019, with full Triple‑A implementation by 2023 ([apnews.com](https://apnews.com/article/9b817bb2c4666c984aa86ed29b401461?utm_source=openai)).
  - **2025 Spring Training**: used in ~60% of games across 13 parks; average of ~4 challenges per game; **52.2% overturn rate**; catchers had the highest success (~56%) ([mlb.com](https://www.mlb.com/news/automated-ball-strike-system-results-mlb-spring-training-2025?utm_source=openai)).
  - **All‑Star Game**: ABS was featured, with several successful challenges ([en.wikipedia.org](https://en.wikipedia.org/wiki/Automated_Ball-Strike_System?utm_source=openai)).

- **Rationale and Reception**  
  The hybrid challenge model was chosen over full automation to preserve the human element of umpiring and catcher framing skills. It also enhances fairness, transparency, and fan engagement, while helping deter game‑fixing and conspiracy theories in an era of widespread sports betting ([nypost.com](https://nypost.com/2025/09/23/sports/mlb-to-officially-begin-robot-umps-era-abs-set-to-be-approved-for-2026/?utm_source=openai)).

---

##  Summary of Key Trends

1. **Enhanced Pitch‑Tracking Precision**  
   Statcast’s upgraded camera system and expanded metrics are delivering unprecedented accuracy and depth in pitch analysis.

2. **Training Revolution**  
   Tools like Rapsodo and TrackMan are transforming pitcher development by providing real‑time, objective feedback.

3. **ABS Challenge System**  
   MLB is set to introduce a hybrid robot‑umpire system in 2026, balancing technological accuracy with traditional umpiring through a challenge-based model.

4. **Strategic and Cultural Shift**  
   The ABS system introduces new strategic layers for players and coaches, while also reflecting MLB’s commitment to innovation, fairness, and fan experience.

---

If you’d like, I can also integrate insights from your uploaded notes or explore how these trends compare to other leagues like the KBO. Just let me know!